In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from scipy.stats import *

from ipywidgets import interact, interactive, fixed, widgets

from bokeh.plotting import figure
from bokeh.io import output_notebook, show#, output_file #Descomentar para guardar el archivo .html
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

In [2]:
from sklearn.preprocessing import StandardScaler
import random as rn

In [3]:
df = pd.read_csv("regLin4.csv")

In [4]:
def agregar_unos(m):
    normalizado = StandardScaler().fit_transform(m)
    unos = np.array([np.ones(normalizado.shape[0])])
    return np.concatenate((unos.T,normalizado),axis=1)

def sets(m):
    datos = agregar_unos(m)
    entrenamiento, prueba = train_test_split(datos, test_size=0.2)
    entrenamiento = np.array(entrenamiento)
    prueba = np.array(prueba)
    return entrenamiento, prueba

def separados(m):
    entrenamiento, prueba = sets(m)
    
    entrenamiento_x = entrenamiento[:,range(0,entrenamiento.shape[1]-1)]
    entrenamiento_y = entrenamiento[:,entrenamiento.shape[1]-1]

    prueba_x = prueba[:,range(0,prueba.shape[1]-1)]
    prueba_y = prueba[:,prueba.shape[1]-1]
    return entrenamiento_x, entrenamiento_y, prueba_x, prueba_y

In [5]:
def escalon(m,w):
    vector = np.array([])
    for i in range(0,m.shape[0]):
        if np.dot(w,m[i]) > 0:
            vector = np.append(vector,1)
        else:
            vector = np.append(vector,-1)
    return vector

In [6]:
def LMS(df,eta):
    """
    Función de eta
    """
    
    lamb = 0
    
    entrenamiento_x, entrenamiento_y, prueba_x, prueba_y = separados(df)
        
    w = np.ones(df.shape[1])

    error = []    

    contador = 0

    while (sum(np.dot(w,entrenamiento_x.T) - entrenamiento_y))**2 > 0.0001:
        error.append((sum(np.dot(w,entrenamiento_x.T) - entrenamiento_y))**2)
        w[0] = w[0] + (eta * sum((entrenamiento_y - np.dot(w,entrenamiento_x.T)))/len(entrenamiento_y))
        for i in range(1,len(w)):
            w[i] = w[i] + (eta * sum((entrenamiento_y - np.dot(w,entrenamiento_x.T)) #
                                     * entrenamiento_x[:,i])/len(entrenamiento_y))
        if (sum(np.dot(w,entrenamiento_x.T) - entrenamiento_y))**2 > error[len(error)-1]:
            break
        contador += 1
    
    error_prueba = (sum(np.dot(w,prueba_x.T) - prueba_y))**2
    
    return error, w, contador, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, error_prueba

In [7]:
def graficar(eta):
        
    error, w, contador, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, error_prueba = LMS(df,eta)
        
    separados = -w[0]/w[1]
    
    vector_entrenamiento = escalon(entrenamiento_x,w)
    vector_prueba = escalon(prueba_x,w)
        
    s1 = figure(width=750, plot_height=500, title=None)
    s1.circle(entrenamiento_x[:,1],vector_entrenamiento, size=4, color="red", alpha=0.5)
    s1.line(np.sort(entrenamiento_x[:,1]), [w[1]*(np.sort(entrenamiento_x[:,1]))[i]+w[0] for i in range(0,len(entrenamiento_x[:,0]))],\
            line_width=1.5, color="navy", alpha=0.7)
    division = Span(location=separados, dimension='height', line_color='green', line_dash='dashed', line_width=1.5)
    s1.add_layout(division)
    s1.title.text = "Perceptron de entrenamiento"
    s1.xaxis.axis_label = "X"
    s1.yaxis.axis_label = "y"
    tab1 = Panel(child=s1, title="Entrenamiento")

    s2 = figure(width=750, plot_height=500, title=None)
    s2.circle(prueba_x[:,1],vector_prueba, size=5, color="red", alpha=0.5)
    s2.line(np.sort(prueba_x[:,1]), [w[1]*(np.sort(prueba_x[:,1]))[i]+w[0] for i in range(0,len(prueba_x[:,0]))], line_width=1.5,\
            color="navy", alpha=0.7)
    division = Span(location=separados, dimension='height', line_color='green', line_dash='dashed', line_width=1.5)
    s2.add_layout(division)
    s2.title.text = "Perceptron de prueba"
    s2.xaxis.axis_label = "X"
    s2.yaxis.axis_label = "y"
    tab2 = Panel(child=s2, title="Prueba")
    

    tabs = Tabs(tabs=[ tab1, tab2])
    
    show(tabs)

interact(graficar, eta=widgets.BoundedFloatText(min=0.0001,max=1,step=0.0001,value=0.05, description='$\eta$: '))

<function __main__.graficar>

# Perceptrón

In [8]:
def perceptron(df,eta):
    
    error, w, contador, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, error_prueba = LMS(df,eta)
    
    error_per = np.array([])
    while abs(sum(entrenamiento_y - escalon(entrenamiento_x,w))) > 0.01:
        for i in range(0,len(w)):
            error_per = np.append(error_per,sum(entrenamiento_y - escalon(entrenamiento_x,w)))
            w[i] = w[i] + eta * np.dot((entrenamiento_y - escalon(entrenamiento_x,w)),entrenamiento_x[:,i])
        
        if abs(sum(entrenamiento_y - escalon(entrenamiento_x,w))) > abs(error_per[len(error_per)-1]):
            break
    
    return error, w, contador, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, error_prueba, error_per

In [9]:
def graficar(eta):
        
    error, w, contador, entrenamiento_x, entrenamiento_y, prueba_x, prueba_y, error_prueba,error_per = perceptron(df,eta)
        
    separados = -w[0]/w[1]
    
    vector_entrenamiento = escalon(entrenamiento_x,w)
    vector_prueba = escalon(prueba_x,w)
        
    s1 = figure(width=750, plot_height=500, title=None)
    s1.circle(entrenamiento_x[:,1],vector_entrenamiento, size=4, color="red", alpha=0.5)
    s1.line(np.sort(entrenamiento_x[:,1]), [w[1]*(np.sort(entrenamiento_x[:,1]))[i]+w[0] for i in range(0,len(entrenamiento_x[:,0]))],\
            line_width=1.5, color="navy", alpha=0.7)
    division = Span(location=separados, dimension='height', line_color='green', line_dash='dashed', line_width=1.5)
    s1.add_layout(division)
    s1.title.text = "Perceptron de entrenamiento"
    s1.xaxis.axis_label = "X"
    s1.yaxis.axis_label = "y"
    tab1 = Panel(child=s1, title="Entrenamiento")

    s2 = figure(width=750, plot_height=500, title=None)
    s2.circle(prueba_x[:,1],vector_prueba, size=5, color="red", alpha=0.5)
    s2.line(np.sort(prueba_x[:,1]), [w[1]*(np.sort(prueba_x[:,1]))[i]+w[0] for i in range(0,len(prueba_x[:,0]))], line_width=1.5,\
            color="navy", alpha=0.7)
    division = Span(location=separados, dimension='height', line_color='green', line_dash='dashed', line_width=1.5)
    s2.add_layout(division)
    s2.title.text = "Perceptron de prueba"
    s2.xaxis.axis_label = "X"
    s2.yaxis.axis_label = "y"
    tab2 = Panel(child=s2, title="Prueba")
    

    tabs = Tabs(tabs=[ tab1, tab2])
    
    show(tabs)

interact(graficar, eta=widgets.BoundedFloatText(min=0.001,max=1,step=0.001,value=0.001, description='$\eta$: '))

<function __main__.graficar>

# La función AND

In [10]:
df_and = pd.DataFrame(np.array([[0,0,0], [0,1,0], [1,0,0], [1,1,1]]))

df_and.columns = ['X1','X2','y']

df_and

,X1,X2,y
0,0,0,0
1,0,1,0
2,1,0,0
3,1,1,1


In [11]:
and_ = np.array([[1,0,0,0], [1,0,1,0], [1,1,0,0], [1,1,1,1]])
and_

array([[1, 0, 0, 0],
       [1, 0, 1, 0],
       [1, 1, 0, 0],
       [1, 1, 1, 1]])

In [12]:
def escalon_cero(m,w):
    vector = np.array([])
    for i in range(0,m.shape[0]):
        if np.dot(w,m[i]) >= 0:
            vector = np.append(vector,1)
        else:
            vector = np.append(vector,0)
    return vector

In [13]:
w = np.ones([df_and.shape[1]])

In [14]:
eta = 0.1

In [15]:
dif = sum(df_and.y - escalon_cero(and_[:,0:3],w))
while abs(dif) != 0.:
    producto = np.dot(w,and_[:,0:3].T)
    for l in xrange(len(w)):
        w[l] = w[l] + (np.dot(eta*(df_and.y - escalon_cero(and_[:,0:3],w)), and_[:,l]))
    dif = sum(df_and.y - escalon_cero(and_[:,0:3],w))

In [16]:
sum(df_and.y - escalon_cero(and_[:,0:3],w))

0.0

In [17]:
escalon_cero(and_[:,0:3],w)

array([ 0.,  0.,  0.,  1.])

In [19]:
intervalo = np.linspace(-0.5, 1.5, 0.1)
p = figure(width=750, plot_height=500, title=None)
p.circle(df_and.X1[(df_and.X1 != df_and.X2) | (df_and.X1==0)],df_and.X2[(df_and.X1 != df_and.X2) | (df_and.X1==0)],size=10, color="red", alpha=0.5) 
p.circle(df_and.X1[(df_and.X1 == df_and.X2) & (df_and.X1==1)],df_and.X2[(df_and.X1 == df_and.X2) & (df_and.X1==1)],size=10, color="blue", alpha=0.5) 
p.line([0,(-w[0]/w[2])+eta], [(-w[0]/w[1])+eta,0], line_width=1.5, color="green", alpha=0.7)
p.title.text = "Perceptron de compuerta lógica AND"
p.xaxis.axis_label = "X1"
p.yaxis.axis_label = "X2"
show(p)  
